In [1]:
import os

In [2]:
%cd C:\Users\anil\Downloads\faster_rcnn_od\fasterrcnn_sickle_cell_detection

C:\Users\anil\Downloads\faster_rcnn_od\fasterrcnn_sickle_cell_detection


In [3]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    params_classes: int
  

In [4]:
from faster_rcnn_od.constants import *
from faster_rcnn_od.utils import read_yaml, create_directories

In [5]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model
        
        create_directories([config.root_dir])
        

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            params_classes=self.params.CLASSES
            
        )
    
        return prepare_base_model_config

In [6]:
import os
import urllib.request as request
from zipfile import ZipFile


In [7]:
import torchvision
import torch
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

In [8]:
class PrepareBaseModel:
    def __init__(self, config: PrepareBaseModelConfig):
        self.config = config

    
    def get_base_model(self):
        self.model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)
        in_features = self.model.roi_heads.box_predictor.cls_score.in_features
        self.model.roi_heads.box_predictor = FastRCNNPredictor(in_features, self.config.params_classes)
       
        
        self.save_model(self.config.base_model_path, self.model)
    

    def save_model(self,path, model):
        model = torch.jit.script(model)
        model.save(path)    




In [9]:
try:
    config = ConfigurationManager()
    prepare_base_model_config = config.get_prepare_base_model_config()
    prepare_base_model = PrepareBaseModel(config=prepare_base_model_config)
    prepare_base_model.get_base_model()
except Exception as e:
    raise e

[2023-05-07 23:24:41,389: INFO: comman: yaml file: config\config.yaml loaded successfully]
[2023-05-07 23:24:41,393: INFO: comman: yaml file: params.yaml loaded successfully]
[2023-05-07 23:24:41,395: INFO: comman: created directory at: artifacts]
[2023-05-07 23:24:41,398: INFO: comman: created directory at: artifacts/prepare_base_model]


c:\Users\anil\anaconda3\envs\fasterrcnn\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\anil\anaconda3\envs\fasterrcnn\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
